<a href="https://colab.research.google.com/github/jayrodge/AudioMNIST-using-PyTorch/blob/master/CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/jayrodge/AudioMNIST-using-PyTorch.git

Cloning into 'AudioMNIST-using-PyTorch'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 60168 (delta 14), reused 13 (delta 1), pack-reused 60128
Receiving objects: 100% (60168/60168), 1.40 GiB | 23.49 MiB/s, done.
Resolving deltas: 100% (17/17), done.
Checking out files: 100% (90006/90006), done.


In [0]:
import os 
os.chdir('AudioMNIST-using-PyTorch/')

# for i in range(10):
#   os.mkdir('MNIST/'+str(i))

In [0]:
import torch
import numpy as np
from tqdm import tqdm
import torch.optim asees optim
from torchvision import datasets
import torchvision.transforms as transforms
from sklearn.model_selection import ShuffleSplit
from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
# how many samples per batch to load
batch_size = 32

# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

In [0]:
train_data = datasets.ImageFolder('MNIST/',transform=transform)

In [0]:
import torch.nn as nn
import torch.nn.functional as F
train_on_gpu = torch.cuda.is_available()
# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer
        self.conv1 = nn.Conv2d(3, 16, 5)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32*53*53, 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, 10)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
    #         x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1, 32 * 53 * 53)
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.softmax(self.fc3(x))
        return x


In [11]:
num_data = len(train_data)
indices_data = list(range(num_data))
ss = ShuffleSplit(n_splits=5, test_size=0.33)

iteration = 0


for train_index, test_index in ss.split(indices_data):
#   print("%s %s" % (len(train_index), len(test_index)))
#   train_idx, test_idx = indices_data[train_index], indices_data[test_index]  
    print("\nIteration ", iteration)
  
    train_sampler = SubsetRandomSampler(train_index)
    test_sampler = SubsetRandomSampler(test_index)

    train_loader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(train_data, sampler=test_sampler, batch_size=batch_size)
    
    classes = [i for i in range(10)]
  
    model = Net().cuda()

    # criterion
    criterion = torch.nn.CrossEntropyLoss()

    # optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum= 0.9)

    n_epochs = 4 # you may increase this number to train a final model

    valid_loss_min = np.Inf # track change in validation loss

    for epoch in tqdm(range(1, n_epochs+1)):

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0


        # train the model 
        model.train()
        for data, target in train_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)

    #Testing

    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    model.eval()
    i=1
    # iterate over test data

    for data, target in test_loader:
        i=i+1
        if len(target)!=batch_size:
            print(i)
            continue
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)    
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class

        for i in range(batch_size):       
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # average test loss
    test_loss = test_loss/len(test_loader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                classes[i], 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    
    iteration = iteration + 1


Iteration  0


100%|██████████| 4/4 [49:38<00:00, 744.66s/it]


32
Test Loss: 0.008580

Test Accuracy of     0: 99% (997/1001)
Test Accuracy of     1: 98% (976/995)
Test Accuracy of     2: 96% (927/958)
Test Accuracy of     3: 99% (973/977)
Test Accuracy of     4: 99% (952/954)
Test Accuracy of     5: 99% (1004/1010)
Test Accuracy of     6: 99% (982/987)
Test Accuracy of     7: 99% (983/988)
Test Accuracy of     8: 99% (1018/1024)
Test Accuracy of     9: 99% (992/994)

Test Accuracy (Overall): 99% (9804/9888)

Iteration  1


100%|██████████| 4/4 [49:37<00:00, 744.22s/it]


32
Test Loss: 0.010754

Test Accuracy of     0: 99% (971/978)
Test Accuracy of     1: 99% (970/977)
Test Accuracy of     2: 98% (949/968)
Test Accuracy of     3: 99% (982/990)
Test Accuracy of     4: 100% (994/994)
Test Accuracy of     5: 99% (982/983)
Test Accuracy of     6: 98% (960/975)
Test Accuracy of     7: 99% (988/991)
Test Accuracy of     8: 99% (1031/1035)
Test Accuracy of     9: 95% (953/997)

Test Accuracy (Overall): 98% (9780/9888)

Iteration  2


...

32
Test Loss: 0.009314

Test Accuracy of     0: 98% (982/996)
Test Accuracy of     1: 99% (1005/1010)
Test Accuracy of     2: 99% (996/997)
Test Accuracy of     3: 99% (975/982)
Test Accuracy of     4: 99% (950/956)
Test Accuracy of     5: 99% (993/1001)
Test Accuracy of     6: 99% (974/983)
Test Accuracy of     7: 97% (998/1019)
Test Accuracy of     8: 98% (939/955)
Test Accuracy of     9: 99% (983/989)

Test Accuracy (Overall): 99% (9795/9888)

Iteration  3


100%|██████████| 4/4 [49:46<00:00, 746.53s/it]


32
Test Loss: 0.008694

Test Accuracy of     0: 99% (949/956)
Test Accuracy of     1: 99% (1016/1025)
Test Accuracy of     2: 99% (979/988)
Test Accuracy of     3: 99% (954/957)
Test Accuracy of     4: 99% (990/995)
Test Accuracy of     5: 98% (954/969)
Test Accuracy of     6: 99% (987/991)
Test Accuracy of     7: 97% (964/990)
Test Accuracy of     8: 99% (988/990)
Test Accuracy of     9: 99% (1018/1027)

Test Accuracy (Overall): 99% (9799/9888)

Iteration  4


100%|██████████| 4/4 [49:48<00:00, 747.04s/it]


32
Test Loss: 0.011487

Test Accuracy of     0: 98% (963/973)
Test Accuracy of     1: 100% (966/966)
Test Accuracy of     2: 99% (999/1001)
Test Accuracy of     3: 98% (985/996)
Test Accuracy of     4: 98% (966/981)
Test Accuracy of     5: 97% (999/1025)
Test Accuracy of     6: 98% (928/939)
Test Accuracy of     7: 99% (1003/1010)
Test Accuracy of     8: 99% (992/993)
Test Accuracy of     9: 98% (985/1004)

Test Accuracy (Overall): 98% (9786/9888)


In [0]:
# Loading saved model
# model.load_state_dict(torch.load('model_cifar.pt'))